In [7]:
import yfinance as yf

ticker = yf.download("BTC-EUR", period="1mo")

print(ticker)

[*********************100%***********************]  1 of 1 completed

Price                         Adj Close         Close          High  \
Ticker                          BTC-EUR       BTC-EUR       BTC-EUR   
Date                                                                  
2024-09-29 00:00:00+00:00  58994.921875  58994.921875  59307.644531   
2024-09-30 00:00:00+00:00  58764.406250  58764.406250  59177.386719   
2024-10-01 00:00:00+00:00  56865.964844  56865.964844  58764.179688   
2024-10-02 00:00:00+00:00  54983.457031  54983.457031  57646.929688   
2024-10-03 00:00:00+00:00  54892.132812  54892.132812  56422.355469   
2024-10-04 00:00:00+00:00  55060.414062  55060.414062  55699.738281   
2024-10-05 00:00:00+00:00  56525.160156  56525.160156  56909.765625   
2024-10-06 00:00:00+00:00  56537.925781  56537.925781  56793.867188   
2024-10-07 00:00:00+00:00  57261.046875  57261.046875  57374.738281   
2024-10-08 00:00:00+00:00  56710.355469  56710.355469  58705.382812   
2024-10-09 00:00:00+00:00  56609.617188  56609.617188  57546.800781   
2024-1

In [6]:
ticker.head()

Price,Adj Close,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,,
2024-09-30 00:00:00+00:00,233.000000,233.000000,233.000000,229.649994,230.039993,54541900
2024-10-01 00:00:00+00:00,226.210007,226.210007,229.649994,223.740005,229.520004,63285000
2024-10-02 00:00:00+00:00,226.779999,226.779999,227.369995,223.020004,225.889999,32880600
2024-10-03 00:00:00+00:00,225.669998,225.669998,226.809998,223.320007,225.139999,34044200
2024-10-04 00:00:00+00:00,226.800003,226.800003,228.000000,224.130005,227.899994,37245100
